# Compromized solution to imitate syncronous/asyncronous BO

In [2]:
import pandas as pd
import numpy as np
import sklearn.gaussian_process as gp
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.gaussian_process.kernels import RBF, Matern
import time
from acquisitions import EI, UCB
from penalizers import LP, HLP
from objectives import branin
from parallel_bo import sample_next_parallel, acq_parallel

In [70]:
def bayesian_optimization(n_iters, function, bounds, acq_func = UCB, penalizer = HLP, local_L = True, n_processes = 4,
                          X_init = None, n_init = 10, gp_params = None, find_min = True, alpha = 1e-5, epsilon = 1e-7, verbose = True):
    
    
    X_tested = []
    y_tested = []
    n_params = bounds.shape[0]
    
    if X_init is None:
        X_init = np.random.uniform(bounds[:,0], bounds[:,1], (n_init, bounds.shape[0]))
    
    for X in X_init:
        X_tested.append(X)
        y_tested.append(function(X))
    
    # numpy array of y needed for processes
    y_np = np.array(y_tested)
    
    # creating the gaussian process
    if gp_params is not None:
        gaussian_process = gp.GaussianProcessRegressor(**args) 
    
    else:
        kernel = Matern()
        gaussian_process = gp.GaussianProcessRegressor(
            kernel = kernel, alpha = alpha, n_restarts_optimizer = 10, normalize_y = True)
        gaussian_process.fit(X_tested, y_tested)

    # Initializing, iteration counts the iteration number
    iteration = 0
    X_eval = X_eval = np.array([[None] * n_params] * n_processes)
    for i in range(n_processes):
        X_eval[i] = sample_next_parallel(acq_parallel, gaussian_process, np.array(X_eval), y_np, bounds, find_min = find_min,
                                                       acq_func = acq_func, penalizer = penalizer, local_L = local_L)
        if verbose:
            print(f'{X_eval[i]} is being evaluated.')
    
        
    for i in range(n_iters):
        
        if verbose:
            print(f'{X_eval[finished]} just finished by worker {finished}.')
            
        finished = np.random.randint(4)
        X_tested.append([val for val in X_eval[finished]])
        y_tested.append(function(X_eval[finished]))
        y_np = np.array(y_tested)
        
        # fit an updated gaussian process
        gaussian_process.fit(X_tested, y_tested)
        
        # then start a new process
        X_eval[finished] = sample_next_parallel(acq_parallel, gaussian_process, X_eval, y_np, bounds, find_min = find_min,
                                                       acq_func = acq_func, penalizer = penalizer, local_L = local_L)
        
        if verbose:
            print(f'{X_eval[finished]} is being evaluated.')
    
        time.sleep(1)

    return X_tested, y_tested


## Sample both LP and HLP for project, then investigate whether locality has positive impact 

In [95]:
names = ['Local LP', 'Local HLP', 'Global LP', 'Global HLP']
results = {}
no_tests = 30

ctr = 0
for local_L in [True, False]:
    
    for penalizer in [LP, HLP]:
        
        for run in range(no_tests):
            X_tested, y_tested = bayesian_optimization(20, branin, bounds = np.array([[-5, 10], [0, 15]]),
                                               n_init = 10, acq_func = UCB, penalizer = penalizer, local_L = local_L, verbose = False)
            
            results[names[ctr] + str(run)] = (np.array(y_tested) * 400 + 400).tolist()
    
        ctr += 1



# minimum = ~ -0.999 with normalization

In [9]:
for row in [[True, 'HLP'], [False, 'LP']]:
    print (row)

[True, 'HLP']
[False, 'LP']
